In [120]:
#https://stackoverflow.com/questions/20716842/python-download-images-from-google-image-search
from hashlib import md5
from urllib.request import Request
from urllib.request import urlopen
import urllib.parse
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import json
import os
import collections as cl

In [202]:
def google_image_search(lang,
                        searchterm ="tomato",
                        json_only=True,
                        limit_search=0,
                        verbose=False):   
    def update_dictionary(d, #d must be a dictionary
                          searchterm,
                          filename,
                          imgurl,
                          siteurl,
                          imgtype,
                          sitetitle,
                          imgtitle,
                          changeflag):
    
        text = '' + imgurl
        m = md5()
        m.update(text.encode('utf-8'))
        
        tmp = {}
        tmp["imgurl"] = imgurl
        tmp["siteurl"] = siteurl
        tmp["imgtype"] = imgtype
        tmp["sitetitle"] = sitetitle
        tmp["imgtitle"] = imgtitle
        tmp["changeflag"] = changeflag
        tmp["hash"] = m.hexdigest()
        d[filename] = tmp
        return d    
    
    if not os.path.exists(lang):
        os.mkdir(lang)
        
    searchdir = searchterm.replace(" ","_")
    searchdir = searchterm.replace("\"","")
    searchterm = urllib.parse.quote(searchterm)
    url = "https://www.google.com/search?gl=us&hl=en&pws=0&gws_rd=cr&q="+searchterm+"&source=lnms&tbm=isch"
    browser = webdriver.Firefox()
    browser.get(url)
    button_xpath = '//input[@id="smb"]'   
    
    for _ in range(500):
        browser.execute_script("window.scrollBy(0,10000)")
        try:
            browser.find_element_by_xpath(button_xpath).click()
        except:
            pass

    #count number of search result
    result = 0
    for x in browser.find_elements_by_xpath('//div[contains(@class,"rg_meta")]'):
        result += 1
    print (result, "images obtainable from this search")
    
    counter = 0
    succounter = 0
    d = dict()
    for x in browser.find_elements_by_xpath('//div[contains(@class,"rg_meta")]'):
        counter += 1
        changeflag = 0

        imgurl = json.loads(x.get_attribute('innerHTML'))["ou"]
        siteurl = json.loads(x.get_attribute('innerHTML'))["ru"]
        imgtype = json.loads(x.get_attribute('innerHTML'))["ity"]
        sitetitle = json.loads(x.get_attribute('innerHTML'))["pt"]
        imgtitle = json.loads(x.get_attribute('innerHTML'))["s"]
        filename = lang + "_" + str(succounter) + "." + imgtype
        
        if verbose == True:
            print ("Total Count:", counter)
            print ("filename",filename)
            print ("imageURL:",imgurl)
            print ("siteURL:",siteurl)
            print ("imgtype:",imgtype)
            print ("sitetitle:",sitetitle)
            print ("imgtitle:",imgtitle)        
        
        try:
            #save image to file when json_only flag is True
            if json_only == False:
                with urllib.request.urlopen(imgurl) as response:
                    raw_img = response.read()      
                File = open(os.path.join(lang,filename), "wb")
                #print (os.path.join(searchdir , filename))
                File.write(raw_img)
                File.close()
                succounter += 1
                if verbose == True:
                    print ("Succsessful Count:", succounter)
                    print ("__________________\n")

            #create and write json object to file
            if imgtype is not None:
                if imgtype not in ["jpg","jpeg","tif","tiff","gif"]:
                    imgtype = "jpg"
                    changeflag = 1
                data = update_dictionary(d,
                                         searchterm,
                                         filename,
                                         imgurl,
                                         siteurl,
                                         imgtype,
                                         sitetitle,
                                         imgtitle,
                                         changeflag)
#                with open(os.path.join(lang,lang+'.json'), 'a') as f:
#                    f.write(data)            
                if json_only == True:
                    succounter += 1
        
        except Exception as e:
            if verbose == True:
                print ("could not retreive the image, and was not recorded to json because of the following reason.")
                print (e)
                print ("__________________\n")

        if limit_search > 0:
            if counter == limit_search:
                break
    with open(os.path.join(lang,lang+'.json'), 'w') as f:
        f.write(json.dumps(d,ensure_ascii=False,indent=True))
    browser.quit()

In [203]:
searchterm = [['ENG','tomato ("alternaria solani" OR "early blight")'],
              ['JPN','トマト ("alternaria solani" OR 輪紋病)'],
              ['CHI','(蕃茄 OR 西红柿) ("alternaria solani" OR 早疫病)'],
              ['FRE','tomate ("alternaria solani" OR alternarienne)']
             ]
for lang, term in searchterm:
    print (lang,term)
    google_image_search(lang = lang,
                        searchterm=term,
                        limit_search=0,
                        json_only=False,
                        verbose=False)

ENG tomato ("alternaria solani" OR "early blight")
787 images obtainable from this search
JPN トマト ("alternaria solani" OR 輪紋病)
1000 images obtainable from this search
CHI (蕃茄 OR 西红柿) ("alternaria solani" OR 早疫病)
958 images obtainable from this search
FRE tomate ("alternaria solani" OR alternarienne)
838 images obtainable from this search
